# Simple ISR Generator

Here's my setup.sh file:


In [1]:
! source /sdf/group/rubin/sw/tag/w_2023_19/loadLSST.bash
! setup lsst_distrib -t w_2023_19

grep: write error: Broken pipe
grep: write error: Broken pipe
grep: write error: Broken pipe
grep: write error: Broken pipe
grep: write error: Broken pipe
grep: write error: Broken pipe
grep: write error: Broken pipe


In [1]:
! eups list -s | grep lsst_distrib
! eups list -s | grep cp_pipe
! eups list -s | grep ip_isr
# lsst_distrib          gdf42428520+d521942ec3 	current w_2023_19 setup

lsst_distrib          g4213664e8e+995cec829e 	w_2023_29 setup
cp_pipe               LOCAL:/sdf/home/a/abrought/alternate_branches/dm_stack/cp_pipe 	setup
ip_isr                LOCAL:/sdf/home/a/abrought/alternate_branches/dm_stack/ip_isr 	setup


In [2]:
import os
import numpy as np
import glob
from astropy.io import fits
import pickle as pkl
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
from astropy.table import Table, vstack, join
from scipy import stats
from scipy.optimize import curve_fit
from lsst.obs.lsst import LsstCam
camera = LsstCam().getCamera()
from matplotlib.pyplot import cm
import lsst.cp.pipe as cpPipe
import lsst.ip.isr as ipIsr
import itertools
from lsst.cp.pipe.makeBrighterFatterKernel import BrighterFatterKernelSolveTask


plt.rcParams.update({'font.size': 15})

slacAmps = {'C10':'AMP01','C11':'AMP02','C12':'AMP03','C13':'AMP04',\
           'C14':'AMP05','C15':'AMP06','C16':'AMP07','C17':'AMP08',\
           'C07':'AMP09','C06':'AMP10','C05':'AMP11','C04':'AMP12',\
           'C03':'AMP13','C02':'AMP14','C01':'AMP15','C00':'AMP16'}

slacAmps2 = {y: x for x, y in slacAmps.items()}

def avg_kernels(d, exclude_channels=[]):
    t = np.zeros(d["C00"].shape)
    for k,channel in zip(d.values(), d.keys()):
        if channel in exclude_channels:
            continue
        t += k
    return t / len(d.values())

In [3]:
#repo_path = "/sdf/group/lsst/camera/IandT/repo_gen3/BOT_data/butler.yaml"

repo_path = "/repo/main"
butler = Butler(repo_path)
registry = butler.registry


In [4]:
# R03-S12 (ITL - Good)
sensor = "R03-S12"
det=23 

In [5]:
# Original good run of PTC

ptc_col = f'u/abrought/BF/2023.07.19/ptc.{sensor}.trunc_to_newpcti.fullnoisematrix'
bfk_col = f'u/abrought/BF/2023.07.19/bfk.{sensor}.20000adu'
bias_col = 'u/cslage/bias_13144'
lin_col = 'u/cslage/linearizer_28jan22/20220128T174703Z'

# Get the PTC
ptc = butler.get('ptc', detector=det, instrument='LSSTCam', collections=ptc_col)

# Get the BFK
bfk = butler.get('bfk', detector=det, instrument='LSSTCam', collections=bfk_col)
bfk_kernel = avg_kernels(bfk.ampKernels)

# Get bias
bias = butler.get('bias', detector=det, instrument="LSSTCam", collections=bias_col)

# Get linearizer
linearizer = butler.get('linearizer', detector=det, instrument="LSSTCam", collections=lin_col)

# Get the dummy exposure
datasetRefs = list(registry.queryDatasets(datasetType="cpPtcExtract", collections=ptc_col))
raw=butler.get('raw', dataId=datasetRefs[100].dataId, collections=ptc_col)

In [11]:
from lsst.ip.isr.isrTask import IsrTask

config = IsrTask.ConfigClass()
config.doBias = True
config.doVariance = True
config.doLinearize = True
config.doCrosstalk = False
config.doDefect = False
config.doNanMasking = True
config.doInterpolate = True
config.doSaturationInterpolation = False
config.doBrighterFatter = False
config.doDark = False
config.doFlat = False
config.doApplyGains = False
config.usePtcGains = True
config.brighterFatterApplyGain = True
config.doFluxConservingBrighterFatterCorrection = True
config.doFringe = False
config.doOverscan = True
config.overscan.fitType = 'MEDIAN_PER_ROW'  
config.doDeferredCharge = False

isrtask = IsrTask(config=config)
result = isrtask.run(raw, camera=camera, bias=bias, linearizer=linearizer,
                     ptc=ptc, bfKernel=bfk_kernel, defects=None, dark=None, 
                     detectorNum=det)


  3.81559732e+04  4.66618013e+04  5.50788275e+04  6.35237786e+04
  7.23547623e+04  8.14810749e+04  0.00000000e+00 -5.31778833e+00
  6.50177541e+00  2.54898913e+01  4.66351121e+01  6.66848427e+01
  5.74596335e+01  1.06545763e+01 -9.41904507e+00  3.62629130e+01]) for C10
  3.64891914e+04  4.46322712e+04  5.26844358e+04  6.07528460e+04
  6.92029638e+04  7.79482682e+04  0.00000000e+00 -5.76840542e+00
  4.56155137e+00  2.31350676e+01  4.41091369e+01  7.15522860e+01
  7.98441089e+01  5.01231165e+01  1.38505590e+01  3.21557682e+01]) for C11
  3.56924305e+04  4.36585512e+04  5.15359580e+04  5.94302472e+04
  6.76883935e+04  7.62552154e+04  0.00000000e+00 -5.06526710e+00
  3.92856102e+00  2.12394314e+01  3.94141906e+01  6.60771703e+01
  7.81098916e+01  5.40256181e+01  1.25948367e+01  8.53181085e+00]) for C12
  3.62595348e+04  4.43511288e+04  5.23523824e+04  6.03697790e+04
  6.87664696e+04  7.74564806e+04  0.00000000e+00 -5.25918145e+00
  4.91174379e+00  2.31556496e+01  4.44626129e+01  7.39919319

In [12]:
result.exposure

# Recreation of Flux Conserving BF ISR Function

In [19]:
import numpy as np
from scipy.signal import convolve2d
import galsim
import matplotlib.pyplot as plt
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath
from scipy.ndimage import gaussian_filter

def transferFlux(cFunc, fStep, correctionMode=True):
    """Take the input convolved deflection potential and the flux array
    to compute and apply the flux transfer into the correction array.
    Parameters
    ----------
    cFunc: `np.array`
        Deflection potential, being the convolution of the flux F with the
        kernel K.
    fStep: `np.array`
        The array of flux values which act as the source of the flux transfer.
    correctionMode: `bool`
        Defines if applying correction (True) or generating sims (False).
    Returns
    -------
    corr:
        BFE correction array
    """

    if cFunc.shape != fStep.shape:
        raise RuntimeError(f'transferFlux: array shapes do not match: {cFunc.shape}, {fStep.shape}')

    # set the sign of the correction and set its value for the
    # time averaged solution
    if correctionMode:
        # negative sign if applying BFE correction
        factor = -0.5
    else:
        # positive sign if generating BFE simulations
        factor = 0.5

    # initialise the BFE correction image to zero
    corr = np.zeros_like(cFunc)

    # Generate a 2D mesh of x,y coordinates
    yDim, xDim = cFunc.shape
    y = np.arange(yDim, dtype=int)
    x = np.arange(xDim, dtype=int)
    xc, yc = np.meshgrid(x, y)

    # process each axis in turn
    for ax in [0, 1]:

        # gradient of phi on right/upper edge of pixel
        diff = np.diff(cFunc, axis=ax)

        # expand array back to full size with zero gradient at the end
        gx = np.zeros_like(cFunc)
        yDiff, xDiff = diff.shape
        gx[:yDiff, :xDiff] += diff

        # select pixels with either positive gradients on the right edge,
        # flux flowing to the right/up
        # or negative gradients, flux flowing to the left/down
        for i, sel in enumerate([gx > 0, gx < 0]):
            xSelPixels = xc[sel]
            ySelPixels = yc[sel]
            # and add the flux into the pixel to the right or top
            # depending on which axis we are handling
            if ax == 0:
                xPix = xSelPixels
                yPix = ySelPixels+1
            else:
                xPix = xSelPixels+1
                yPix = ySelPixels
            # define flux as the either current pixel value or pixel
            # above/right
            # depending on whether positive or negative gradient
            if i == 0:
                # positive gradients, flux flowing to higher coordinate values
                flux = factor * fStep[sel]*gx[sel]
            else:
                # negative gradients, flux flowing to lower coordinate values
                flux = factor * fStep[yPix, xPix]*gx[sel]
            # change the fluxes of the donor and receiving pixels
            # such that flux is conserved
            corr[sel] -= flux
            corr[yPix, xPix] += flux

    # return correction array
    return corr

def applyFluxConservingC18(kernel, img, maxIter=100, threshold=10):
    # Implemented by Lance Miller

    # get kernel and its shape
    kLy, kLx = kernel.shape
    kernelImage = afwImage.ImageD(kLx, kLy)
    kernelImage.getArray()[:, :] = kernel
    image = afwImage.ImageD(img.shape[1], img.shape[0])
    image.getArray()[:, :] = img
    tempImage = image.clone()


    nanIndex = np.isnan(tempImage.getArray())
    tempImage.getArray()[nanIndex] = 0.

    outImage = afwImage.ImageF(image.getDimensions())
    corr = np.zeros_like(image.getArray())
    prevImage = np.zeros_like(image.getArray())
    
    # IF second parameter is true, correction diverges
    convCntrl = afwMath.ConvolutionControl(False, False, 1)

    fixedKernel = afwMath.FixedKernel(kernelImage)

    # set the padding amount
    # ensure we pad by an even amount larger than the kernel
    kLy = 2 * ((1+kLy)//2)
    kLx = 2 * ((1+kLx)//2)

    # The deflection potential only depends on the gradient of
    # the convolution, so we can subtract the mean, which then
    # allows us to pad the image with zeros and avoid wrap-around effects
    # (although still not handling the image edges with a physical model)
    # This wouldn't be great if there were a strong image gradient.
    imydim, imxdim = tempImage.array.shape
    imean = np.mean(tempImage.getArray()[~nanIndex])
    # subtract mean from image
    tempImage -= imean
    tempImage.array[nanIndex] = 0.
    padArray = np.pad(tempImage.getArray(), ((0, kLy), (0, kLx)))

    outImage = afwImage.ImageF(np.pad(outImage.getArray(), ((0, kLy), (0, kLx))))
    
    # Convert array to afw image so afwMath.convolve works
    padImage = afwImage.ImageF(padArray.shape[1], padArray.shape[0])
    padImage.array[:] = padArray

    for iteration in range(maxIter):
        print(f'{iteration + 1}/{maxIter}')
        
        # create deflection potential, convolution of flux with kernel
        # using padded counts array
        
        afwMath.convolve(outImage, padImage, fixedKernel, convCntrl)
        tmpArray = tempImage.getArray()
        outArray = outImage.getArray()
        #print(outArray)

        # trim convolution output back to original shape
        outArray = outArray[:imydim, :imxdim]

        # generate the correction array, with correctionMode set as input
        corr[...] = transferFlux(outArray, tmpArray, correctionMode=True)

        # update the arrays for the next iteration
        tmpArray[:, :] = image.getArray()[:, :]
        tmpArray += corr
        tmpArray[nanIndex] = 0.
        # update padded array
        # subtract mean
        tmpArray -= imean
        tempImage.array[nanIndex] = 0.
        padArray = np.pad(tempImage.getArray(), ((0, kLy), (0, kLx)))

        if iteration > 0:
            diff = np.sum(np.abs(prevImage - tmpArray))
            print(diff)
            if diff < threshold:
                break
            prevImage[:, :] = tmpArray[:, :]

    image.getArray()[:] += corr[:]
    
    return image.getArray(), iteration

# The image needs to be units of electrons/holes

def applyC18(kernel, img, maxIter=100, threshold = 10.):

    kLx = np.shape(kernel)[1]
    kLy = np.shape(kernel)[0]
    kernelImage = afwImage.ImageD(kLx, kLy)
    kernelImage.getArray()[:, :] = kernel
    image = afwImage.ImageD(img.shape[1], img.shape[0])
    image.getArray()[:, :] = img
    tempImage = image.clone()

    nanIndex = np.isnan(tempImage.getArray())
    tempImage.getArray()[nanIndex] = 0.

    outImage = afwImage.ImageD(image.getDimensions())
    corr = np.zeros_like(image.getArray())
    prev_image = np.zeros_like(image.getArray())
    convCntrl = afwMath.ConvolutionControl(False, True, 1)
    fixedKernel = afwMath.FixedKernel(kernelImage)

    # Define boundary by convolution region.  The region that the
    # correction will be calculated for is one fewer in each dimension
    # because of the second derivative terms.
    # NOTE: these need to use integer math, as we're using start:end as
    # np index ranges.
    startX = kLx//2
    endX = -kLx//2
    startY = kLy//2
    endY = -kLy//2
    

    for iteration in range(maxIter):
        #print(iteration)
        #print(outImage.getArray().shape, tempImage.getArray().shape, fixedKernel.getArray.shape())
        afwMath.convolve(outImage, tempImage, fixedKernel, convCntrl)
        #print(outImage.getArray().shape, tempImage.getArray().shape, fixedKernel.getArray.shape())
        tmpArray = tempImage.getArray()
        outArray = outImage.getArray()

        with np.errstate(invalid="ignore", over="ignore"):
            # First derivative term
            gradTmp = np.gradient(tmpArray[startY:endY, startX:endX])
            gradOut = np.gradient(outArray[startY:endY, startX:endX])
            first = (gradTmp[0]*gradOut[0] + gradTmp[1]*gradOut[1])[1:-1, 1:-1]

            # Second derivative term
            diffOut20 = np.diff(outArray, 2, 0)[startY:endY, startX + 1:endX - 1]
            diffOut21 = np.diff(outArray, 2, 1)[startY + 1:endY - 1, startX:endX]
            second = tmpArray[startY + 1:endY - 1, startX + 1:endX - 1]*(diffOut20 + diffOut21)

            corr[startY + 1:endY - 1, startX + 1:endX - 1] = 0.5*(first + second)

            tmpArray[:, :] = image.getArray()[:, :]
            tmpArray[nanIndex] = 0.
            tmpArray[startY:endY, startX:endX] += corr[startY:endY, startX:endX]

        #if iteration > 0:
        diff = np.sum(np.abs(prev_image - tmpArray))
        print(iteration, diff)
        if diff < threshold:
            break
        prev_image[:, :] = tmpArray[:, :]

    image.getArray()[startY + 1:endY - 1, startX + 1:endX - 1] += corr[startY + 1:endY - 1, startX + 1:endX - 1]
    
    return image.getArray(), iteration


In [16]:
# Diverges if the convolution control copy edge is True
outImage, iteration = applyC18(bfk_kernel, result.outputExposure.getImage().getArray())

1/100
2/100
226885133.20367005
3/100
0.0012210356095181396


In [20]:
# Diverges if the convolution control copy edge is True
outImage, iteration = applyC18(bfk_kernel, result.outputExposure.getImage().getArray())

0 4729129702.04525
1 14.920870366005929
2 0.0075290975278221595


In [21]:
226885133 - 4729129702

-4502244569